In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
yr = "2013-14"

yr[5:]

In [ ]:
for yearStr in [
    "2013-14",
    "2014-15",
    "2015-16",
    "2016-17",
    "2017-18",
    "2018-19",
    "2020-21",
    "2021-22",
    "2022-23",
]:
    exec(f"trny{yearStr[5:]}Rnd1 = pd.read_csv('../data/{yearStr}_combined-stats.csv')")

# Above iteration returns dataframe of team stats for each year's tourney teams at the time of round 1
# → → trny14Rnd1, trny15Rnd1, trny16Rnd1, trny17Rnd1, ..., trny22Rnd1, trny23Rnd1

In [ ]:
for year in [2014, 2015, 2016, 2017, 2018, 2019, 2021, 2022, 2023]:
    for round, res in [(2, 6), (3, 5), (4, 4), (5, 3), (6, 2)]:
        yr = str(year)[2:]
        exec(f"trny{yr}Rnd{round} = trny{yr}Rnd1[trny{yr}Rnd1['trnyRes'] <= {res}]")

# Above iteration returns dataframe of team stats for each year's tourney teams at the time of each round following the first
# → → trny14Rnd2, trny14Rnd3, trny14Rnd4, trny14Rnd5, ..., trny23Rnd5, trny23Rnd6

In [ ]:
for round in [1, 2, 3, 4, 5, 6]:
    exec(
        f"rnd{round}Merge = pd.concat([trny14Rnd{round}, trny15Rnd{round}, trny16Rnd{round}, trny17Rnd{round}, trny18Rnd{round}, trny19Rnd{round}, trny21Rnd{round}, trny22Rnd{round}, trny23Rnd{round}])"
    )

# Above iteration returns dataframe of team stats for all years combined in each round
# → → rnd1Merge, rnd2Merge, rnd3Merge, rnd4Merge, rnd5Merge, rnd6Merge

In [ ]:
# Variables to be used in each scatter plot (baseline from round 1)

offEffMeanRnd1 = np.mean(rnd1Merge["adjOffEff"])
offEffStdRnd1 = np.std(rnd1Merge["adjOffEff"])
defEffMeanRnd1 = np.mean(rnd1Merge["adjDefEff"])
defEffStdRnd1 = np.std(rnd1Merge["adjDefEff"])

rnd1xMin = defEffMeanRnd1 - (1 * defEffStdRnd1)
rnd1xMax = defEffMeanRnd1 + (1 * defEffStdRnd1)
rnd1yMin = offEffMeanRnd1 - (1 * offEffStdRnd1)
rnd1yMax = offEffMeanRnd1 + (1 * offEffStdRnd1)

In [ ]:
ax = plt.subplots()

def calcScopedEffStats(df):
    defEffMean = np.mean(df["adjDefEff"])
    defEffStd = np.std(df["adjDefEff"])
    offEffMean = np.mean(df["adjOffEff"])
    offEffStd = np.std(df["adjOffEff"])

    return defEffMean, defEffStd, offEffMean, offEffStd

def calcScopedBounds(df):
    defEffMean, defEffStd, offEffMean, offEffStd = calcScopeEffStats(df)
    xMin = defEffMean - (1 * defEffStd)
    xMax = defEffMean + (1 * defEffStd)
    yMin = offEffMean - (1 * offEffStd)
    yMax = offEffMean + (1 * offEffStd)
    
    return xMin, xMax, yMin, yMax


def renderFigure(bgColor, rnd):
    ax.set_facecolor(bgColor)
    ax.set_title(f"2024 Offense/Defense Efficiencies ({rnd})")
    ax.set_xlabel("Adjusted Defensive Efficiency")
    ax.set_ylabel("Adjusted Offensive Efficiency")


def renderBaseBound():
    ax.axvline(x=dEffAvgRnd1, color="white", linestyle="-", linewidth=0.5)
    ax.axhline(y=oEffAvgRnd1, color="white", linestyle="-", linewidth=0.5)
    ax.fill_between(
        x=[xMinRnd1, xMaxRnd1],
        y1=yMinRnd1,
        y2=yMaxRnd1,
        color="white",
        alpha=0.25,
        linewidth=0,
    )


def renderScopedBound(df):
    xMin, xMax, yMin, yMax = calcScopedBounds(df)
    ax.axvline(x=xMin, color="orangered", linestyle="-.", linewidth=0.625)
    ax.axvline(x=xMax, color="orangered", linestyle="-.", linewidth=0.625)
    ax.axhline(y=yMin, color="orangered", linestyle="-.", linewidth=0.625)
    ax.axhline(y=yMax, color="orangered", linestyle="-.", linewidth=0.625)
    ax.fill_between(
        x=[xMin, xMax],
        y1=yMin,
        y2=yMax,
        color="orangered",
        alpha=0.25,
    )

def renderScatter(df, ptColor, ptSize, edgeColors, ptLabels):
    ax.scatter(
        df["adjDefEff"],
        df["adjOffEff"],
        c=ptColor,
        s=ptSize,
        edgecolors=edgeColors
    )
    
    if ptLabels:
        for _, point in df.iterrows():
            ax.text(point["adjDefEff"] - 0.125, point["adjOffEff"] + 0.25, point["trnySeed"], fontsize=8, ha="right", va="bottom", c="white")

        for _, point in df.iterrows():
            ax.text(point["adjDefEff"] + 0.125, point["adjOffEff"] + 0.25, point["team"], fontsize=8, ha="left", va="bottom", c="white")


def plotScatter(df, bgColor, rnd, ptColor, ptSize, edgeColors="none", ptLabels=False):
    renderFigure(bgColor, rnd)
    renderBaseBound()
    renderScopedBound(df)
    renderScatter(df, ptColor, ptSize, edgeColors, ptLabels)
    plt.show()

dfRnd1 = rnd1Merge
bgColorRnd1 = "slategray"
rndRnd1 = "Rnd of 64"
ptColorRnd1 = ["lightcyan" if x == 1 else "yellow" if 1 < x <= 6 else "lime" for x in dfRnd1["trnyRes"]]
ptSizeRnd1 = 5
edgeColorsRnd1 = ["lime" if x == 1 else "none" for x in dfRnd1["trnyRes"]]


def plotScatter(df=dfRnd1, bgColor=bgColorRnd1, rnd=rndRnd1, ptColor=ptColorRnd1, ptSize=ptSizeRnd1, edgeColors=edgeColorsRnd1, ptLabels=False):
    renderFigure(bgColor, rnd)
    renderBaseBound()
    renderScopedBound(df)
    renderScatter(df, ptColor, ptSize, edgeColors, ptLabels)
    plt.show()


# fig, ax = plt.subplots()

# ax.set_facecolor("slategray")
# ax.set_title("2014-2023 Offense/Defense Efficiencies (Rnd of 64)")
# ax.set_xlabel("Adjusted Defensive Efficiency")
# ax.set_ylabel("Adjusted Offensive Efficiency")

# ax.axvline(x=(rnd1xMinScoped), color="orangered", linestyle="-.", linewidth=0.625)
# ax.axvline(x=(rnd1xMaxScoped), color="orangered", linestyle="-.", linewidth=0.625)
# ax.axhline(y=(rnd1yMinScoped), color="orangered", linestyle="-.", linewidth=0.625)
# ax.axhline(y=(rnd1yMaxScoped), color="orangered", linestyle="-.", linewidth=0.625)
# ax.fill_between(
#     x=[rnd1xMinScoped, rnd1xMaxScoped],
#     y1=rnd1yMinScoped,
#     y2=rnd1yMaxScoped,
#     color="orangered",
#     alpha=0.25,
# )

# ax.axvline(x=rnd1DefEffMean, color="white", linestyle="-", linewidth=0.5)
# ax.axhline(y=rnd1OffEffMean, color="white", linestyle="-", linewidth=0.5)
# ax.fill_between(
#     x=[rnd1xMin, rnd1xMax],
#     y1=rnd1yMin,
#     y2=rnd1yMax,
#     color="white",
#     alpha=0.25,
#     linewidth=0,
# )

# ax.scatter(
#     rnd1Merge["adjDefEff"],
#     rnd1Merge["adjOffEff"],
#     c=[
#         "lime" if x == 1 else "yellow" if 1 < x <= 6 else "lightcyan"
#         for x in rnd1Merge["trnyRes"]
#     ],
#     edgecolors=["lime" if x == 1 else "none" for x in rnd1Merge["trnyRes"]],
#     s=5,
# )

# plt.show()

In [ ]:
trny24 = pd.read_csv('../data/2023-24_tourney_teams.csv')

oEffAvgRnd1 = np.mean(trny24["adjOffEff"])
oEffStdRnd1 = np.std(trny24["adjOffEff"])
dEffAvgRnd1 = np.mean(trny24["adjDefEff"])
dEffStdRnd1 = np.std(trny24["adjDefEff"])

xMinRnd1 = dEffAvgRnd1 - (1 * dEffStdRnd1)
xMaxRnd1 = dEffAvgRnd1 + (1 * dEffStdRnd1)
yMinRnd1 = oEffAvgRnd1 - (1 * oEffStdRnd1)
yMaxRnd1 = oEffAvgRnd1 + (1 * oEffStdRnd1)

t24Rnd1xMinScoped = dEffAvgRnd1 - (1.75 * dEffStdRnd1)
t24Rnd1xMaxScoped = dEffAvgRnd1 + (0.5 * dEffStdRnd1)
t24Rnd1yMinScoped = oEffAvgRnd1 - (0.75 * oEffStdRnd1)
t24Rnd1yMaxScoped = oEffAvgRnd1 + (2 * oEffStdRnd1)


In [ ]:
rnd2OffEffMean = np.mean(rnd2Merge["adjOffEff"])
rnd2OffEffStd = np.std(rnd2Merge["adjOffEff"])
rnd2DefEffMean = np.mean(rnd2Merge["adjDefEff"])
rnd2DefEffStd = np.std(rnd2Merge["adjDefEff"])

rnd2xMin = rnd2DefEffMean - (1 * rnd2DefEffStd)
rnd2xMax = rnd2DefEffMean + (1 * rnd2DefEffStd)
rnd2yMin = rnd2OffEffMean - (1 * rnd2OffEffStd)
rnd2yMax = rnd2OffEffMean + (1 * rnd2OffEffStd)

rnd2xMinScoped = rnd2DefEffMean - (1.5 * rnd2DefEffStd)
rnd2xMaxScoped = rnd2DefEffMean + (0.75 * rnd2DefEffStd)
rnd2yMinScoped = rnd2OffEffMean - (0.75 * rnd2OffEffStd)
rnd2yMaxScoped = rnd2OffEffMean + (2 * rnd2OffEffStd)

fig, ax = plt.subplots()

ax.set_facecolor("slategray")
ax.set_title("2014-2023 Offense/Defense Efficiencies (Rnd of 32)")
ax.set_xlabel("Adjusted Defensive Efficiency")
ax.set_ylabel("Adjusted Offensive Efficiency")

ax.axvline(x=rnd2xMinScoped, color="orangered", linestyle="-.", linewidth=0.625)
ax.axvline(x=rnd2xMaxScoped, color="orangered", linestyle="-.", linewidth=0.625)
ax.axhline(y=rnd2yMinScoped, color="orangered", linestyle="-.", linewidth=0.625)
ax.axhline(y=rnd2yMaxScoped, color="orangered", linestyle="-.", linewidth=0.625)
ax.fill_between(
    x=[rnd2xMinScoped, rnd2xMaxScoped],
    y1=rnd2yMinScoped,
    y2=rnd2yMaxScoped,
    color="orangered",
    alpha=0.25,
)

ax.axvline(x=rnd2DefEffMean, color="dodgerblue", linestyle="-", linewidth=0.5)
ax.axhline(y=rnd2OffEffMean, color="dodgerblue", linestyle="-", linewidth=0.5)
ax.fill_between(
    x=[rnd2xMin, rnd2xMax],
    y1=rnd2yMin,
    y2=rnd2yMax,
    color="dodgerblue",
    alpha=0.25,
    linewidth=0,
)

ax.axvline(x=rnd1DefEffMean, color="white", linestyle="-", linewidth=0.5)
ax.axhline(y=rnd1OffEffMean, color="white", linestyle="-", linewidth=0.5)
ax.fill_between(
    x=[rnd1xMin, rnd1xMax],
    y1=rnd1yMin,
    y2=rnd1yMax,
    color="white",
    alpha=0.25,
    linewidth=0,
)

ax.scatter(
    rnd2Merge["adjDefEff"],
    rnd2Merge["adjOffEff"],
    c=[
        "lime" if x == 1 else "yellow" if 1 < x <= 5 else "lightcyan"
        for x in rnd2Merge["trnyRes"]
    ],
    edgecolors=["lime" if x == 1 else "none" for x in rnd2Merge["trnyRes"]],
    s=7.5,
)

plt.show()

In [ ]:
rnd3OffEffMean = np.mean(rnd3Merge["adjOffEff"])
rnd3OffEffStd = np.std(rnd3Merge["adjOffEff"])
rnd3DefEffMean = np.mean(rnd3Merge["adjDefEff"])
rnd3DefEffStd = np.std(rnd3Merge["adjDefEff"])

rnd3xMin = rnd3DefEffMean - (1 * rnd3DefEffStd)
rnd3xMax = rnd3DefEffMean + (1 * rnd3DefEffStd)
rnd3yMin = rnd3OffEffMean - (1 * rnd3OffEffStd)
rnd3yMax = rnd3OffEffMean + (1 * rnd3OffEffStd)

rnd3xMinScoped = rnd3DefEffMean - (1.5 * rnd3DefEffStd)
rnd3xMaxScoped = rnd3DefEffMean + (0.75 * rnd3DefEffStd)
rnd3yMinScoped = rnd3OffEffMean - (0.75 * rnd3OffEffStd)
rnd3yMaxScoped = rnd3OffEffMean + (2 * rnd3OffEffStd)

fig, ax = plt.subplots()

ax.set_facecolor("slategray")
ax.set_title("2014-2023 Offense/Defense Efficiencies (Sweet 16)")
ax.set_xlabel("Adjusted Defensive Efficiency")
ax.set_ylabel("Adjusted Offensive Efficiency")

ax.axvline(x=rnd3xMinScoped, color="orangered", linestyle="-.", linewidth=0.625)
ax.axvline(x=rnd3xMaxScoped, color="orangered", linestyle="-.", linewidth=0.625)
ax.axhline(y=rnd3yMinScoped, color="orangered", linestyle="-.", linewidth=0.625)
ax.axhline(y=rnd3yMaxScoped, color="orangered", linestyle="-.", linewidth=0.625)
ax.fill_between(
    x=[rnd3xMinScoped, rnd3xMaxScoped],
    y1=rnd3yMinScoped,
    y2=rnd3yMaxScoped,
    color="orangered",
    alpha=0.25,
)

ax.axvline(x=rnd3DefEffMean, color="dodgerblue", linestyle="-", linewidth=0.5)
ax.axhline(y=rnd3OffEffMean, color="dodgerblue", linestyle="-", linewidth=0.5)
ax.fill_between(
    x=[rnd3xMin, rnd3xMax],
    y1=rnd3yMin,
    y2=rnd3yMax,
    color="dodgerblue",
    alpha=0.25,
    linewidth=0,
)

ax.axvline(x=rnd1DefEffMean, color="white", linestyle="-", linewidth=0.5)
ax.axhline(y=rnd1OffEffMean, color="white", linestyle="-", linewidth=0.5)
ax.fill_between(
    x=[rnd1xMin, rnd1xMax],
    y1=rnd1yMin,
    y2=rnd1yMax,
    color="white",
    alpha=0.25,
    linewidth=0,
)

ax.scatter(
    rnd3Merge["adjDefEff"],
    rnd3Merge["adjOffEff"],
    c=[
        "lime" if x == 1 else "yellow" if 1 < x <= 4 else "lightcyan"
        for x in rnd3Merge["trnyRes"]
    ],
    edgecolors=["lime" if x == 1 else "none" for x in rnd3Merge["trnyRes"]],
    s=10,
)

plt.show()

In [ ]:
rnd4OffEffMean = np.mean(rnd4Merge["adjOffEff"])
rnd4OffEffStd = np.std(rnd4Merge["adjOffEff"])
rnd4DefEffMean = np.mean(rnd4Merge["adjDefEff"])
rnd4DefEffStd = np.std(rnd4Merge["adjDefEff"])

rnd4xMin = rnd4DefEffMean - (1 * rnd4DefEffStd)
rnd4xMax = rnd4DefEffMean + (1 * rnd4DefEffStd)
rnd4yMin = rnd4OffEffMean - (1 * rnd4OffEffStd)
rnd4yMax = rnd4OffEffMean + (1 * rnd4OffEffStd)

rnd4xMinScoped = rnd4DefEffMean - (1 * rnd4DefEffStd)
rnd4xMaxScoped = rnd4DefEffMean + (0.75 * rnd4DefEffStd)
rnd4yMinScoped = rnd4OffEffMean - (1 * rnd4OffEffStd)
rnd4yMaxScoped = rnd4OffEffMean + (2 * rnd4OffEffStd)

fig, ax = plt.subplots()

ax.set_facecolor("slategray")
ax.set_title("2014-2023 Offense/Defense Efficiencies (Elite 8)")
ax.set_xlabel("Adjusted Defensive Efficiency")
ax.set_ylabel("Adjusted Offensive Efficiency")

ax.axvline(x=rnd4xMinScoped, color="orangered", linestyle="-.", linewidth=0.625)
ax.axvline(x=rnd4xMaxScoped, color="orangered", linestyle="-.", linewidth=0.625)
ax.axhline(y=rnd4yMinScoped, color="orangered", linestyle="-.", linewidth=0.625)
ax.axhline(y=rnd4yMaxScoped, color="orangered", linestyle="-.", linewidth=0.625)
ax.fill_between(
    x=[rnd4xMinScoped, rnd4xMaxScoped],
    y1=rnd4yMinScoped,
    y2=rnd4yMaxScoped,
    color="orangered",
    alpha=0.25,
)

ax.axvline(x=rnd4DefEffMean, color="dodgerblue", linestyle="-", linewidth=0.5)
ax.axhline(y=rnd4OffEffMean, color="dodgerblue", linestyle="-", linewidth=0.5)
ax.fill_between(
    x=[rnd4xMin, rnd4xMax],
    y1=rnd4yMin,
    y2=rnd4yMax,
    color="dodgerblue",
    alpha=0.25,
    linewidth=0,
)

ax.axvline(x=rnd1DefEffMean, color="white", linestyle="-", linewidth=0.5)
ax.axhline(y=rnd1OffEffMean, color="white", linestyle="-", linewidth=0.5)
ax.fill_between(
    x=[rnd1xMin, rnd1xMax],
    y1=rnd1yMin,
    y2=rnd1yMax,
    color="white",
    alpha=0.25,
    linewidth=0,
)

ax.scatter(
    rnd4Merge["adjDefEff"],
    rnd4Merge["adjOffEff"],
    c=[
        "lime" if x == 1 else "yellow" if 1 < x <= 3 else "lightcyan"
        for x in rnd4Merge["trnyRes"]
    ],
    edgecolors=["lime" if x == 1 else "none" for x in rnd4Merge["trnyRes"]],
    s=12.5,
)

plt.show()

In [ ]:
rnd5OffEffMean = np.mean(rnd5Merge["adjOffEff"])
rnd5OffEffStd = np.std(rnd5Merge["adjOffEff"])
rnd5DefEffMean = np.mean(rnd5Merge["adjDefEff"])
rnd5DefEffStd = np.std(rnd5Merge["adjDefEff"])

rnd5xMin = rnd5DefEffMean - (1 * rnd5DefEffStd)
rnd5xMax = rnd5DefEffMean + (1 * rnd5DefEffStd)
rnd5yMin = rnd5OffEffMean - (1 * rnd5OffEffStd)
rnd5yMax = rnd5OffEffMean + (1 * rnd5OffEffStd)

rnd5xMinScoped = rnd5DefEffMean - (1 * rnd5DefEffStd)
rnd5xMaxScoped = rnd5DefEffMean + (0.5 * rnd5DefEffStd)
rnd5yMinScoped = rnd5OffEffMean - (0 * rnd5OffEffStd)
rnd5yMaxScoped = rnd5OffEffMean + (2 * rnd5OffEffStd)

fig, ax = plt.subplots()

ax.set_facecolor("slategray")
ax.set_title("2014-2023 Offense/Defense Efficiencies (Final 4)")
ax.set_xlabel("Adjusted Defensive Efficiency")
ax.set_ylabel("Adjusted Offensive Efficiency")

ax.axvline(x=rnd5xMinScoped, color="orangered", linestyle="-.", linewidth=0.625)
ax.axvline(x=rnd5xMaxScoped, color="orangered", linestyle="-.", linewidth=0.625)
ax.axhline(y=rnd5yMinScoped, color="orangered", linestyle="-.", linewidth=0.625)
ax.axhline(y=rnd5yMaxScoped, color="orangered", linestyle="-.", linewidth=0.625)
ax.fill_between(
    x=[rnd5xMinScoped, rnd5xMaxScoped],
    y1=rnd5yMinScoped,
    y2=rnd5yMaxScoped,
    color="orangered",
    alpha=0.25,
)

ax.axvline(x=rnd5DefEffMean, color="dodgerblue", linestyle="-", linewidth=0.5)
ax.axhline(y=rnd5OffEffMean, color="dodgerblue", linestyle="-", linewidth=0.5)
ax.fill_between(
    x=[rnd5xMin, rnd5xMax],
    y1=rnd5yMin,
    y2=rnd5yMax,
    color="dodgerblue",
    alpha=0.25,
    linewidth=0,
)

ax.axvline(x=rnd1DefEffMean, color="white", linestyle="-", linewidth=0.5)
ax.axhline(y=rnd1OffEffMean, color="white", linestyle="-", linewidth=0.5)
ax.fill_between(
    x=[rnd1xMin, rnd1xMax],
    y1=rnd1yMin,
    y2=rnd1yMax,
    color="white",
    alpha=0.25,
    linewidth=0,
)

ax.scatter(
    rnd5Merge["adjDefEff"],
    rnd5Merge["adjOffEff"],
    c=[
        "lime" if x == 1 else "yellow" if 1 < x <= 2 else "lightcyan"
        for x in rnd5Merge["trnyRes"]
    ],
    edgecolors=["lime" if x == 1 else "none" for x in rnd5Merge["trnyRes"]],
    s=15,
)

plt.show()

In [ ]:
rnd6OffEffMean = np.mean(rnd6Merge["adjOffEff"])
rnd6OffEffStd = np.std(rnd6Merge["adjOffEff"])
rnd6DefEffMean = np.mean(rnd6Merge["adjDefEff"])
rnd6DefEffStd = np.std(rnd6Merge["adjDefEff"])

rnd6xMin = rnd6DefEffMean - (1 * rnd6DefEffStd)
rnd6xMax = rnd6DefEffMean + (1 * rnd6DefEffStd)
rnd6yMin = rnd6OffEffMean - (1 * rnd6OffEffStd)
rnd6yMax = rnd6OffEffMean + (1 * rnd6OffEffStd)

rnd6xMinScoped = rnd6DefEffMean - (1 * rnd6DefEffStd)
rnd6xMaxScoped = rnd6DefEffMean + (0.5 * rnd6DefEffStd)
rnd6yMinScoped = rnd6OffEffMean - (0.5 * rnd6OffEffStd)
rnd6yMaxScoped = rnd6OffEffMean + (1 * rnd6OffEffStd)

fig, ax = plt.subplots()

ax.set_facecolor("slategray")
ax.set_title("2014-2023 Offense/Defense Efficiencies (Championship)")
ax.set_xlabel("Adjusted Defensive Efficiency")
ax.set_ylabel("Adjusted Offensive Efficiency")

ax.axvline(x=rnd6xMinScoped, color="orangered", linestyle="-.", linewidth=0.625)
ax.axvline(x=rnd6xMaxScoped, color="orangered", linestyle="-.", linewidth=0.625)
ax.axhline(y=rnd6yMinScoped, color="orangered", linestyle="-.", linewidth=0.625)
ax.axhline(y=rnd6yMaxScoped, color="orangered", linestyle="-.", linewidth=0.625)
ax.fill_between(
    x=[rnd6xMinScoped, rnd6xMaxScoped],
    y1=rnd6yMinScoped,
    y2=rnd6yMaxScoped,
    color="orangered",
    alpha=0.25,
)

ax.axvline(x=rnd6DefEffMean, color="dodgerblue", linestyle="-", linewidth=0.5)
ax.axhline(y=rnd6OffEffMean, color="dodgerblue", linestyle="-", linewidth=0.5)
ax.fill_between(
    x=[rnd6xMin, rnd6xMax],
    y1=rnd6yMin,
    y2=rnd6yMax,
    color="dodgerblue",
    alpha=0.25,
    linewidth=0,
)

ax.axvline(x=rnd1DefEffMean, color="white", linestyle="-", linewidth=0.5)
ax.axhline(y=rnd1OffEffMean, color="white", linestyle="-", linewidth=0.5)
ax.fill_between(
    x=[rnd1xMin, rnd1xMax],
    y1=rnd1yMin,
    y2=rnd1yMax,
    color="white",
    alpha=0.25,
    linewidth=0,
)

ax.scatter(
    rnd6Merge["adjDefEff"],
    rnd6Merge["adjOffEff"],
    c=[
        "lime" if x == 1 else "lightcyan"
        for x in rnd6Merge["trnyRes"]
    ],
    edgecolors=["lime" if x == 1 else "none" for x in rnd6Merge["trnyRes"]],
    s=17.5,
)

plt.show()

In [ ]:
trny24 = pd.read_csv('../data/2023-24_tourney_teams.csv')

oEffAvgRnd1 = np.mean(trny24["adjOffEff"])
oEffStdRnd1 = np.std(trny24["adjOffEff"])
dEffAvgRnd1 = np.mean(trny24["adjDefEff"])
dEffStdRnd1 = np.std(trny24["adjDefEff"])

xMinRnd1 = dEffAvgRnd1 - (1 * dEffStdRnd1)
xMaxRnd1 = dEffAvgRnd1 + (1 * dEffStdRnd1)
yMinRnd1 = oEffAvgRnd1 - (1 * oEffStdRnd1)
yMaxRnd1 = oEffAvgRnd1 + (1 * oEffStdRnd1)

t24Rnd1xMinScoped = dEffAvgRnd1 - (1.75 * dEffStdRnd1)
t24Rnd1xMaxScoped = dEffAvgRnd1 + (0.5 * dEffStdRnd1)
t24Rnd1yMinScoped = oEffAvgRnd1 - (0.75 * oEffStdRnd1)
t24Rnd1yMaxScoped = oEffAvgRnd1 + (2 * oEffStdRnd1)

ax = plt.subplots()

def renderBackground(bgColor, rnd):
    ax.set_facecolor(bgColor)
    ax.set_title(f"2024 Offense/Defense Efficiencies ({rnd})")
    ax.set_xlabel("Adjusted Defensive Efficiency")
    ax.set_ylabel("Adjusted Offensive Efficiency")


def renderBaseBndArea():
    ax.axvline(x=dEffAvgRnd1, color="white", linestyle="-", linewidth=0.5)
    ax.axhline(y=oEffAvgRnd1, color="white", linestyle="-", linewidth=0.5)
    ax.fill_between(
        x=[xMinRnd1, xMaxRnd1],
        y1=yMinRnd1,
        y2=yMaxRnd1,
        color="white",
        alpha=0.25,
        linewidth=0,
    )


def renderScopeBndArea(xMin, xMax, yMin, yMax):
    ax.axvline(x=xMin, color="orangered", linestyle="-.", linewidth=0.625)
    ax.axvline(x=xMax, color="orangered", linestyle="-.", linewidth=0.625)
    ax.axhline(y=yMin, color="orangered", linestyle="-.", linewidth=0.625)
    ax.axhline(y=yMax, color="orangered", linestyle="-.", linewidth=0.625)
    ax.fill_between(
        x=[xMin, xMax],
        y1=yMin,
        y2=yMax,
        color="orangered",
        alpha=0.25,
    )

def renderScatter(df, ptColor, ptSize, edgeColors, ptLabels):
    ax.scatter(
        df["adjDefEff"],
        df["adjOffEff"],
        c=ptColor,
        s=ptSize,
        edgecolors=edgeColors,
        s=20,
    )
    
    if ptLabels:
        for _, point in df.iterrows():
            ax.text(point["adjDefEff"] - 0.125, point["adjOffEff"] + 0.25, point["trnySeed"], fontsize=8, ha="right", va="bottom", c="white")

        for _, point in df.iterrows():
            ax.text(point["adjDefEff"] + 0.125, point["adjOffEff"] + 0.25, point["team"], fontsize=8, ha="left", va="bottom", c="white")


def plotScatter(df, bgColor, rnd, xMin, xMax, yMin, yMax, ptColor, ptSize, edgeColors="none", ptLabels=False):
    renderBackground(rnd)
    renderBaseBndArea()
    renderScopeBndArea(xMin, xMax, yMin, yMax)
    renderScatter(df, ptColor, ptSize, edgeColors, ptLabels)
    plt.show()

# ax.set_facecolor("slategray")
# ax.set_title("2024 Offense/Defense Efficiencies (Rnd of 64)")
# ax.set_xlabel("Adjusted Defensive Efficiency")
# ax.set_ylabel("Adjusted Offensive Efficiency")

# ax.axvline(x=(t24Rnd1xMinScoped), color="orangered", linestyle="-.", linewidth=0.625)
# ax.axvline(x=(t24Rnd1xMaxScoped), color="orangered", linestyle="-.", linewidth=0.625)
# ax.axhline(y=(t24Rnd1yMinScoped), color="orangered", linestyle="-.", linewidth=0.625)
# ax.axhline(y=(t24Rnd1yMaxScoped), color="orangered", linestyle="-.", linewidth=0.625)
# ax.fill_between(
#     x=[t24Rnd1xMinScoped, t24Rnd1xMaxScoped],
#     y1=t24Rnd1yMinScoped,
#     y2=t24Rnd1yMaxScoped,
#     color="orangered",
#     alpha=0.25,
# )

# ax.axvline(x=t24Rnd1DefEffMean, color="white", linestyle="-", linewidth=0.5)
# ax.axhline(y=t24Rnd1OffEffMean, color="white", linestyle="-", linewidth=0.5)
# ax.fill_between(
#     x=[t24Rnd1xMin, t24Rnd1xMax],
#     y1=t24Rnd1yMin,
#     y2=t24Rnd1yMax,
#     color="white",
#     alpha=0.25,
#     linewidth=0,
# )

# ax.scatter(
#     tourney2024["adjDefEff"],
#     tourney2024["adjOffEff"],
#     c="yellow",
#     s=5,
# )

# plt.show()

In [ ]:
inOffRange = tourney2024["adjOffEff"].between(t24Rnd1yMinScoped, t24Rnd1yMaxScoped)
tourney2024[inOffRange]

In [ ]:
inDefRange = tourney2024["adjDefEff"].between(t24Rnd1xMinScoped, t24Rnd1xMaxScoped)
tourney2024[inDefRange]

In [ ]:
offRange = tourney2024[inOffRange]
defRange = tourney2024[inDefRange]

inBothRanges = tourney2024[inOffRange & inDefRange]
inBothRanges

In [ ]:
tourney2024East = tourney2024[tourney2024["trnyReg"] == "East"]

fig, ax = plt.subplots()

ax.set_facecolor("slategray")
ax.set_title("2024 Offense/Defense Efficiencies (East - Rnd 1)")
ax.set_xlabel("Adjusted Defensive Efficiency")
ax.set_ylabel("Adjusted Offensive Efficiency")

ax.axvline(x=(t24Rnd1xMinScoped), color="orangered", linestyle="-.", linewidth=0.625)
ax.axvline(x=(t24Rnd1xMaxScoped), color="orangered", linestyle="-.", linewidth=0.625)
ax.axhline(y=(t24Rnd1yMinScoped), color="orangered", linestyle="-.", linewidth=0.625)
ax.axhline(y=(t24Rnd1yMaxScoped), color="orangered", linestyle="-.", linewidth=0.625)
ax.fill_between(
    x=[t24Rnd1xMinScoped, t24Rnd1xMaxScoped],
    y1=t24Rnd1yMinScoped,
    y2=t24Rnd1yMaxScoped,
    color="orangered",
    alpha=0.25,
)

ax.axvline(x=t24Rnd1DefEffMean, color="white", linestyle="-", linewidth=0.5)
ax.axhline(y=t24Rnd1OffEffMean, color="white", linestyle="-", linewidth=0.5)
ax.fill_between(
    x=[t24Rnd1xMin, t24Rnd1xMax],
    y1=t24Rnd1yMin,
    y2=t24Rnd1yMax,
    color="white",
    alpha=0.25,
    linewidth=0,
)

ax.scatter(
    tourney2024East["adjDefEff"],
    tourney2024East["adjOffEff"],
    c=[
        "lime" if x <= 8 else "tomato"
        for x in tourney2024East["trnySeed"]
    ],
    s=20,
)

for i, point in tourney2024East.iterrows():
    ax.text(point["adjDefEff"] - 0.125, point["adjOffEff"] + 0.25, point["trnySeed"], fontsize=8, ha="right", va="bottom", c="white")

for i, point in tourney2024East.iterrows():
    ax.text(point["adjDefEff"] + 0.125, point["adjOffEff"] + 0.25, point["team"], fontsize=8, ha="left", va="bottom", c="white")

plt.show()

In [ ]:
tourney2024West = tourney2024[tourney2024["trnyReg"] == "West"]

fig, ax = plt.subplots()

ax.set_facecolor("slategray")
ax.set_title("2024 Offense/Defense Efficiencies (West - Rnd 1)")
ax.set_xlabel("Adjusted Defensive Efficiency")
ax.set_ylabel("Adjusted Offensive Efficiency")

ax.axvline(x=(t24Rnd1xMinScoped), color="orangered", linestyle="-.", linewidth=0.625)
ax.axvline(x=(t24Rnd1xMaxScoped), color="orangered", linestyle="-.", linewidth=0.625)
ax.axhline(y=(t24Rnd1yMinScoped), color="orangered", linestyle="-.", linewidth=0.625)
ax.axhline(y=(t24Rnd1yMaxScoped), color="orangered", linestyle="-.", linewidth=0.625)
ax.fill_between(
    x=[t24Rnd1xMinScoped, t24Rnd1xMaxScoped],
    y1=t24Rnd1yMinScoped,
    y2=t24Rnd1yMaxScoped,
    color="orangered",
    alpha=0.25,
)

ax.axvline(x=t24Rnd1DefEffMean, color="white", linestyle="-", linewidth=0.5)
ax.axhline(y=t24Rnd1OffEffMean, color="white", linestyle="-", linewidth=0.5)
ax.fill_between(
    x=[t24Rnd1xMin, t24Rnd1xMax],
    y1=t24Rnd1yMin,
    y2=t24Rnd1yMax,
    color="white",
    alpha=0.25,
    linewidth=0,
)

ax.scatter(
    tourney2024West["adjDefEff"],
    tourney2024West["adjOffEff"],
    c=[
        "lime" if x <= 8 else "tomato"
        for x in tourney2024West["trnySeed"]
    ],
    edgecolors=["purple" if x == 16 else "none" for x in tourney2024West["trnySeed"]],
    s=20,
)

for i, point in tourney2024West.iterrows():
    ax.text(point["adjDefEff"] - 0.125, point["adjOffEff"] + 0.25, point["trnySeed"], fontsize=8, ha="right", va="bottom", c="white")

for i, point in tourney2024West.iterrows():
    ax.text(point["adjDefEff"] + 0.125, point["adjOffEff"] + 0.25, point["team"], fontsize=8, ha="left", va="bottom", c="white")

plt.show()

In [ ]:
tourney2024South = tourney2024[tourney2024["trnyReg"] == "South"]

fig, ax = plt.subplots()

ax.set_facecolor("slategray")
ax.set_title("2024 Offense/Defense Efficiencies (South - Rnd 1)")
ax.set_xlabel("Adjusted Defensive Efficiency")
ax.set_ylabel("Adjusted Offensive Efficiency")

ax.axvline(x=(t24Rnd1xMinScoped), color="orangered", linestyle="-.", linewidth=0.625)
ax.axvline(x=(t24Rnd1xMaxScoped), color="orangered", linestyle="-.", linewidth=0.625)
ax.axhline(y=(t24Rnd1yMinScoped), color="orangered", linestyle="-.", linewidth=0.625)
ax.axhline(y=(t24Rnd1yMaxScoped), color="orangered", linestyle="-.", linewidth=0.625)
ax.fill_between(
    x=[t24Rnd1xMinScoped, t24Rnd1xMaxScoped],
    y1=t24Rnd1yMinScoped,
    y2=t24Rnd1yMaxScoped,
    color="orangered",
    alpha=0.25,
)

ax.axvline(x=t24Rnd1DefEffMean, color="white", linestyle="-", linewidth=0.5)
ax.axhline(y=t24Rnd1OffEffMean, color="white", linestyle="-", linewidth=0.5)
ax.fill_between(
    x=[t24Rnd1xMin, t24Rnd1xMax],
    y1=t24Rnd1yMin,
    y2=t24Rnd1yMax,
    color="white",
    alpha=0.25,
    linewidth=0,
)

ax.scatter(
    tourney2024South["adjDefEff"],
    tourney2024South["adjOffEff"],
    c=[
        "lime" if x <= 8 else "tomato"
        for x in tourney2024South["trnySeed"]
    ],
    edgecolors=["purple" if x == 10 else "none" for x in tourney2024South["trnySeed"]],
    s=20,
)

for i, point in tourney2024South.iterrows():
    ax.text(point["adjDefEff"] - 0.125, point["adjOffEff"] + 0.25, point["trnySeed"], fontsize=8, ha="right", va="bottom", c="white")

for i, point in tourney2024South.iterrows():
    ax.text(point["adjDefEff"] + 0.125, point["adjOffEff"] + 0.25, point["team"], fontsize=8, ha="left", va="bottom", c="white")

plt.show()

In [ ]:
tourney2024Midwest = tourney2024[tourney2024["trnyReg"] == "Midwest"]

fig, ax = plt.subplots()

ax.set_facecolor("slategray")
ax.set_title("2024 Offense/Defense Efficiencies (Midwest - Rnd 1)")
ax.set_xlabel("Adjusted Defensive Efficiency")
ax.set_ylabel("Adjusted Offensive Efficiency")

ax.axvline(x=(t24Rnd1xMinScoped), color="orangered", linestyle="-.", linewidth=0.625)
ax.axvline(x=(t24Rnd1xMaxScoped), color="orangered", linestyle="-.", linewidth=0.625)
ax.axhline(y=(t24Rnd1yMinScoped), color="orangered", linestyle="-.", linewidth=0.625)
ax.axhline(y=(t24Rnd1yMaxScoped), color="orangered", linestyle="-.", linewidth=0.625)
ax.fill_between(
    x=[t24Rnd1xMinScoped, t24Rnd1xMaxScoped],
    y1=t24Rnd1yMinScoped,
    y2=t24Rnd1yMaxScoped,
    color="orangered",
    alpha=0.25,
)

ax.axvline(x=t24Rnd1DefEffMean, color="white", linestyle="-", linewidth=0.5)
ax.axhline(y=t24Rnd1OffEffMean, color="white", linestyle="-", linewidth=0.5)
ax.fill_between(
    x=[t24Rnd1xMin, t24Rnd1xMax],
    y1=t24Rnd1yMin,
    y2=t24Rnd1yMax,
    color="white",
    alpha=0.25,
    linewidth=0,
)

ax.scatter(
    tourney2024Midwest["adjDefEff"],
    tourney2024Midwest["adjOffEff"],
    c=[
        "lime" if x <= 8 else "tomato"
        for x in tourney2024Midwest["trnySeed"]
    ],
    edgecolors=["purple" if x == 10 or x == 16 else "none" for x in tourney2024Midwest["trnySeed"]],
    s=20,
)

for i, point in tourney2024Midwest.iterrows():
    ax.text(point["adjDefEff"] - 0.125, point["adjOffEff"] + 0.25, point["trnySeed"], fontsize=8, ha="right", va="bottom", c="white")

for i, point in tourney2024Midwest.iterrows():
    ax.text(point["adjDefEff"] + 0.125, point["adjOffEff"] + 0.25, point["team"], fontsize=8, ha="left", va="bottom", c="white")

plt.show()